In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
! pip install -q kaggle

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json



Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 70 bytes


In [11]:

!kaggle datasets download "ai-generated-images-vs-real-images/AiArtData/AiArtData"

403 - Forbidden - Permission 'datasets.get' was denied


In [12]:


import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

# DATAPATH
ai_image_dir = r"/kaggle/input/ai-generated-images-vs-real-images/AiArtData/AiArtData"
real_image_dir = r"/kaggle/input/ai-generated-images-vs-real-images/RealArt/RealArt"

# FUNCTION TO PREPROCESS LOAD AND IMAGE DATA
def preprocess_image(image_path, target_size=(128, 128)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, target_size)
    image = tf.image.random_flip_left_right(image)  # Randomly flip the image horizontally
    image = tf.image.random_flip_up_down(image)     # Randomly flip the image vertically
    image = tf.image.random_brightness(image, max_delta=0.2)  # Randomly adjust brightness
    image = tf.image.random_contrast(image, lower=0.2, upper=1.8)  # Randomly adjust contrast
    image = tf.image.random_saturation(image, lower=0.2, upper=1.8)  # Randomly adjust saturation
    image = tf.image.random_hue(image, max_delta=0.1)  # Randomly adjust hue
    image = tf.clip_by_value(image, 0, 255)  # Clip pixel values to [0, 255]
    image = image / 255.0  # Normalize pixel values to [0, 1]
    return image

# FUNCTION TO LOAD IMAGE FROM DIR AND PREPROCES THEM
def load_and_preprocess_images(image_dir):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir)]
    images = [preprocess_image(image_path) for image_path in image_paths]
    return images

# LOAD AND PREPROCESS AI GENERATED IMAGE
ai_images = load_and_preprocess_images(ai_image_dir)
ai_labels = np.ones(len(ai_images))  # Set label 1 for AI-generated images

# LOAD AND PREPROCESS REAL IMAGE
real_images = load_and_preprocess_images(real_image_dir)
real_labels = np.zeros(len(real_images))  # Set label 0 for real images

# COMBINE AI GENERATED AND REAL IMAGE
images = ai_images + real_images
labels = np.concatenate([ai_labels, real_labels])

# CONVERT LABEL TO ONE HOT ENCODING
encoder = OneHotEncoder(sparse=False)
labels = encoder.fit_transform(labels.reshape(-1, 1))

# CONVERT IMAGE TO LABEL AND NUMPY ARRAYS
images = np.array(images)
labels = np.array(labels)

# SHUFFLE IMAGE AND LABELS
indices = np.arange(len(images))
np.random.shuffle(indices)
images = images[indices]
labels = labels[indices]

#SHOW PREPROCESS IMAGE
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(images[i])
    plt.title("AI" if np.argmax(labels[i]) == 1 else "Real")
    plt.axis('off')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/ai-generated-images-vs-real-images/AiArtData/AiArtData'